# Run Ceph with 2 OSDs and Run Benchmark to Compare Latencies of Ceph using:
#### 1. Librados
#### 2. Rados Block Device (RBD)
#### 3. Rados Gateway (RGW)

## 1. Setup

In [1]:
# Create Reservation
    # https://chi.tacc.chameleoncloud.org/project/
    
    # 1. Reserve Physical Host
        # click "Leases" => "+ Create Lease"
        # lease name = "trovi-ceph"
            # "$node_type", "compute_skylake"
            # Max lease: 
                # 7 days
    # 2. Reserve Network [No-NEED]

# Launching an Instance
    # In the sidebar, click Compute, then click Instances
    # Click on the Launch Instance
        # pick the correct reservation 
        # count = 2 (ex name: ceph-node-1, ceph-node-2)
        # Image: "CC-CentOS7"    # need 1062?? no
    # Select "sharednet1"
    # Choose the ssh key

# Allocate floating IPs
    # Book the IP interface
        # Click "Network -> Floating IPs -> Allocate IP To Project"
        # Write description
        # Click "Allocate IP"

    # click "Associate" OR click "attach interface"
        # Click "Network -> Floating IPs"

    # ony 6 available public interfaces
    # wait a few minutes
    # is it Spawning?? https://chi.tacc.chameleoncloud.org/project/instances/

# If you can't access the cc user on SSH
    # open the console terminal via the website
    # edit the .ssh/authorized_keys and add your pub_key manually!!!
    # now, the hostname will be different 
        # e.g. from dan-storage to zhenz-test
    # run "sudo dhclient" from the web-based ssh console

In [2]:
# export basic ssh variables
export SSHKEY_FILE="$HOME/.ssh/$USER-trovi-ceph"
export SSHKEY_NAME="$USER-trovi-ceph"

In [3]:
# create ssh folder 
mkdir -p ~/.ssh
echo "Host *" > ~/.ssh/config
echo " StrictHostKeyChecking no" >> ~/.ssh/config
sudo rm -rf output/*

# create ssh key-pair
rm -rf $HOME/.ssh/$USER-*
ssh-keygen -t rsa -b 4096  -P '' -C $SSHKEY_NAME -f $SSHKEY_FILE <<< y
echo "SSH key with name: $SSHKEY_NAME created"

Generating public/private rsa key pair.
Your identification has been saved in /home/rani_api3939_gmail_com/.ssh/rani_api3939_gmail_com-trovi-ceph
Your public key has been saved in /home/rani_api3939_gmail_com/.ssh/rani_api3939_gmail_com-trovi-ceph.pub
The key fingerprint is:
SHA256:ZbTGndsKlymIc7lz0mMVjDQ8KhfXOHne7Mu8oCkHpx0 rani_api3939_gmail_com-trovi-ceph
The key's randomart image is:
+---[RSA 4096]----+
|         .++     |
|        .+B*o.   |
|         +B==o   |
|      ..o*  .*o  |
|      ooS o *..  |
|       o.oE= ..  |
|        +==.oo . |
|        o=o+ .+  |
|         oo   .. |
+----[SHA256]-----+
SSH key with name: rani_api3939_gmail_com-trovi-ceph created


In [4]:
# Copy the public key 
pubkey=`cat $SSHKEY_FILE.pub`
echo "echo \"$pubkey\" >> ~/.ssh/authorized_keys"
echo "chmod 644 ~/.ssh/authorized_keys"
echo "echo 'StrictHostKeyChecking no' > ~/.ssh/config; chmod go-rw ~/.ssh/config"

# Add it to the reserved node (Do it for both instances)
# 1. Open your local terminal
# 2. Connect to the node (ssh cc@xxx.xx.xx) as if you are connecting to the normal chameleon node
# 3. Copy and run this printed command  

echo "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDeMyr5YUCusz/7mWLhtYdIcAOjwcUu80U88eDh99TgivTC5vPaLa+UtFGcI6PndSIgw12ZIEIeRin/kC9QYBEnAP7XPQPdTUn0JIKXRZyzGoyi2iVzR8ci4z2qDTJpbLCAJgKGBtPypXaVyqyQ9m4AJNCV3OZWfk4gx0sEgx7nIpj2rbaa0iqWBx9aYeqKHlPBFpjctLD8sOWChfBHWTTdZMh8tS4pNlHt7ZNoZv/6J+g/zGkMLqXFkYerSPVsKObyCji7VlElH++TGNCMDF9zraHbpA8r3kpFIPcTtQwmLfgyDTebWJLNz2htWr4izPiUPOULs+kXSDyrG7PMTQCobqt6sOKhVqLHuUR0p+0/T0xLxc7QDjkxdEgUYzH+eevC+0EOIn43c+MKjL75AXyzxf45U0vLJl+fTWRjXOv5HM4oZ0y8nvWN+D595hQnnJXJKeZI7QD3aidT+F7U9Pk9UNujpcwkeafw0/IUz3ulJWWP1VyAQB/hmYE51CB6aLIBj4g5jtWY3yi9SrIJoX3UveKsC/XsNaJjiLz0qP84oPG5p8Gckjb2u70BfUxVrMNdUB+tCp3WmFgtlCyd6YJjz45wTPpk4eaur/WxhGW42KCCpGuPpB6LYVK6qkmIaHMLBivVhMgAZh4tb+Wo060PZmtT4b2ZdUFAsikRE67HQQ== rani_api3939_gmail_com-trovi-ceph" >> ~/.ssh/authorized_keys
chmod 644 ~/.ssh/authorized_keys
echo 'StrictHostKeyChecking no' > ~/.ssh/config; chmod go-rw ~/.ssh/config


## 2. Set Login Command [Do this after restart]

#### If the ip address has ever used before in this session (ex. after rebuild), run below command

In [5]:
eval "$login_command_1" /bin/bash << EOF
    ssh-keygen -f "/home/rani_api3939_gmail_com/.ssh/known_hosts" -R "192.5.86.149"
EOF
eval "$login_command_1" /bin/bash << EOF
    ssh-keygen -f "/home/rani_api3939_gmail_com/.ssh/known_hosts" -R "192.5.86.216"
EOF

# Host 192.5.86.149 found: line 2
# Host 192.5.86.149 found: line 4
# Host 192.5.86.149 found: line 5
# Host 192.5.86.149 found: line 6
/home/rani_api3939_gmail_com/.ssh/known_hosts updated.
Original contents retained as /home/rani_api3939_gmail_com/.ssh/known_hosts.old
# Host 192.5.86.216 found: line 2
# Host 192.5.86.216 found: line 3
# Host 192.5.86.216 found: line 4
# Host 192.5.86.216 found: line 5
/home/rani_api3939_gmail_com/.ssh/known_hosts updated.
Original contents retained as /home/rani_api3939_gmail_com/.ssh/known_hosts.old


#### IMPORTANT: change the value of server1_ip and server2_ip below!

In [6]:
export server1_ip="192.5.86.149"
export server2_ip="192.5.86.216"
login_command_1="ssh -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE cc@$server1_ip"
login_command_2="ssh -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE cc@$server2_ip"
eval "$login_command_1" pwd 
eval "$login_command_2" pwd 
wait_ssh "$server1_ip"
wait_ssh "$server2_ip"

/home/cc
/home/cc
Waiting up to 300 seconds for SSH on 192.5.86.149...
SSH is running!
Waiting up to 300 seconds for SSH on 192.5.86.216...
SSH is running!


#### Testing connection

In [7]:
eval "$login_command_1" /bin/bash << EOF
    pwd
    ls
EOF
eval "$login_command_2" /bin/bash << EOF
    pwd
    ls
EOF

# Expected output:
    # /home/cc
    # my_mounting_point
    # openrc
    # README

/home/cc
my_mounting_point
openrc
README
/home/cc
my_mounting_point
openrc
README


In [8]:
ssh cc@"$server1_ip" \
    "realpath . && ls"
ssh cc@"$server2_ip" \
    "realpath . && ls"

/home/cc
my_mounting_point
openrc
README
/home/cc
my_mounting_point
openrc
README


## 3. Create Loop Devices

#### Check if the instance already has loop devices

In [9]:
eval "$login_command_1" /bin/bash << EOF
    lsblk
EOF
eval "$login_command_2" /bin/bash << EOF
    lsblk
EOF

# Expected output
# NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
# sda      8:0    0 223.6G  0 disk 
# ├─sda1   8:1    0   550M  0 part /boot/efi
# ├─sda2   8:2    0     8M  0 part 
# └─sda3   8:3    0   223G  0 part /
# NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
# sda      8:0    0 223.6G  0 disk 
# ├─sda1   8:1    0   550M  0 part /boot/efi
# ├─sda2   8:2    0     8M  0 part 
# └─sda3   8:3    0   223G  0 part /

NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
sda      8:0    0 223.6G  0 disk 
├─sda1   8:1    0   550M  0 part /boot/efi
├─sda2   8:2    0     8M  0 part 
└─sda3   8:3    0   223G  0 part /
NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
sda      8:0    0 223.6G  0 disk 
├─sda1   8:1    0   550M  0 part /boot/efi
├─sda2   8:2    0     8M  0 part 
└─sda3   8:3    0   223G  0 part /


#### If the output shows that there are loop devices already, no need to run below 2 cells

In [10]:
eval "$login_command_1" /bin/bash << EOF
    sudo chown cc -R /mnt/
EOF
eval "$login_command_2" /bin/bash << EOF
    sudo chown cc -R /mnt/
EOF

In [11]:
eval "$login_command_1" /bin/bash << EOF
    # https://www.thegeekdiary.com/how-to-create-virtual-block-device-loop-device-filesystem-in-linux/
    # If there is only 1 physical storage, you must create loop devices!
        # linux support block device called the loop device, which maps a normal file onto a virtual block device

    # Create a file (25 GB each) [Run-Once]

        mkdir -p /mnt/extra/loop-files/
        cd /mnt/extra/loop-files/
        dd if=/dev/zero of=loopbackfile1.img bs=100M count=250
        cp loopbackfile1.img loopbackfile2.img
        cp loopbackfile1.img loopbackfile3.img
        cp loopbackfile1.img loopbackfile4.img
        cp loopbackfile1.img loopbackfile5.img

    # Create the loop device  [Run AFTER EACH REBOOT]
        cd /mnt/extra/loop-files/
        sudo losetup -fP loopbackfile1.img
        sudo losetup -fP loopbackfile2.img
        sudo losetup -fP loopbackfile3.img
        sudo losetup -fP loopbackfile4.img
        sudo losetup -fP loopbackfile5.img

    # print loop devices  [Run AFTER EACH REBOOT]
        losetup -a
            # /dev/loop0: []: (/mnt/extra/loop-files/loopbackfile1.img)
            # /dev/loop1: []: (/mnt/extra/loop-files/loopbackfile2.img)
            # /dev/loop2: []: (/mnt/extra/loop-files/loopbackfile3.img)

    # Format devices into filesystems  [Run AFTER EACH REBOOT]
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile1.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile2.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile3.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile4.img 
        printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile5.img 
EOF

eval "$login_command_2" /bin/bash << EOF
    # https://www.thegeekdiary.com/how-to-create-virtual-block-device-loop-device-filesystem-in-linux/
    # If there is only 1 physical storage, you must create loop devices!
        # linux support block device called the loop device, which maps a normal file onto a virtual block device

    # Create a file (25 GB each) [Run-Once]

        mkdir -p /mnt/extra/loop-files/
        cd /mnt/extra/loop-files/
        dd if=/dev/zero of=loopbackfile1.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile2.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile3.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile4.img bs=100M count=250
        dd if=/dev/zero of=loopbackfile5.img bs=100M count=250

    # Create the loop device  [Run AFTER EACH REBOOT]
        cd /mnt/extra/loop-files/
        sudo losetup -fP loopbackfile1.img
        sudo losetup -fP loopbackfile2.img
        sudo losetup -fP loopbackfile3.img
        sudo losetup -fP loopbackfile4.img
        sudo losetup -fP loopbackfile5.img

    # print loop devices  [Run AFTER EACH REBOOT]
        losetup -a
            # /dev/loop0: []: (/mnt/extra/loop-files/loopbackfile1.img)
            # /dev/loop1: []: (/mnt/extra/loop-files/loopbackfile2.img)
            # /dev/loop2: []: (/mnt/extra/loop-files/loopbackfile3.img)

    # Format devices into filesystems  [Run AFTER EACH REBOOT]
        printf "y" | sudo mkfs -t xfs -q /mnt/extra/loop-files/loopbackfile1.img 
        printf "y" | sudo mkfs -t xfs -q /mnt/extra/loop-files/loopbackfile2.img 
        printf "y" | sudo mkfs -t xfs -q /mnt/extra/loop-files/loopbackfile3.img 
        printf "y" | sudo mkfs -t xfs -q /mnt/extra/loop-files/loopbackfile4.img 
        printf "y" | sudo mkfs -t xfs -q /mnt/extra/loop-files/loopbackfile5.img 
        lsblk -f
EOF

# Expected output
# NAME   FSTYPE LABEL           UUID                                 MOUNTPOINT
# sda                                                                
# ├─sda1 vfat   MKFS_ESP        6E49-AC05                            /boot/efi
# ├─sda2                                                             
# └─sda3 ext4   cloudimg-rootfs a67cccae-b568-4133-a688-6d25d911f291 /
# loop0                                                              
# loop1                                                              
# loop2                                                              
# loop3                                                              
# loop4 

250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 18.9604 s, 1.4 GB/s
/bin/bash: line 24: losetup: command not found
/mnt/extra/loop-files/loopbackfile1.img is not a block special device.
Proceed anyway? (y,n) mke2fs 1.42.9 (28-Dec-2013)
Discarding device blocks:    4096/6400002101248/6400004198400/6400006295552/640000              done                            
Filesystem label=
OS type: Linux
Block size=4096 (log=2)
Fragment size=4096 (log=2)
Stride=0 blocks, Stripe width=0 blocks
1602496 inodes, 6400000 blocks
320000 blocks (5.00%) reserved for the super user
First data block=0
Maximum filesystem blocks=2155872256
196 block groups
32768 blocks per group, 32768 fragments per group
8176 inodes per group
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000

Allocating group tables:   0/19      done                            
Writing inode tables:   0/19      done                            
Creati

## 4. Install Ceph

In [12]:
eval "$login_command_1" /bin/bash << EOF
    echo y | sudo yum update
    sudo yum makecache
    sudo yum -y install subscription-manager
    sudo yum -y install firewalld
EOF

eval "$login_command_2" /bin/bash << EOF
    echo y | sudo yum update
    sudo yum makecache
    sudo yum -y install subscription-manager
    sudo yum -y install firewalld
EOF

Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: mirrors.cmich.edu
 * epel: mirror.dst.ca
 * extras: mirror.grid.uchicago.edu
 * updates: mirrors.cmich.edu
http://mirrors.cmich.edu/centos/7.9.2009/os/x86_64/repodata/repomd.xml: [Errno 12] Timeout on http://mirrors.cmich.edu/centos/7.9.2009/os/x86_64/repodata/repomd.xml: (28, 'Operation too slow. Less than 1000 bytes/sec transferred the last 30 seconds')
Trying other mirror.
http://mirror.es.its.nyu.edu/epel/7/x86_64/repodata/repomd.xml: [Errno 12] Timeout on http://mirror.es.its.nyu.edu/epel/7/x86_64/repodata/repomd.xml: (28, 'Connection timed out after 30001 milliseconds')
Trying other mirror.
Resolving Dependencies
--> Running transaction check
---> Package ansible.noarch 0:2.9.25-1.el7 will be updated
---> Package ansible.noarch 0:2.9.27-1.el7 will be an update
---> Package at.x86_64 0:3.1.13-24.el7 will be updated
---> Package at.x86_64 0:3.1.13-25.el7_9 will be an update
---> Package bind-export-libs.x86_64 32:9.

#### Add Ceph Repo

In [13]:
ssh cc@"$server1_ip" \
    "cat << EOF | sudo tee /etc/yum.repos.d/ceph.repo
[ceph-noarch]
name=Ceph noarch packages
baseurl=https://download.ceph.com/rpm-luminous/el7/noarch
enabled=1
gpgcheck=1
type=rpm-md
gpgkey=https://download.ceph.com/keys/release.asc
EOF"

ssh cc@"$server2_ip" \
    "cat << EOF | sudo tee /etc/yum.repos.d/ceph.repo
[ceph-noarch]
name=Ceph noarch packages
baseurl=https://download.ceph.com/rpm-luminous/el7/noarch
enabled=1
gpgcheck=1
type=rpm-md
gpgkey=https://download.ceph.com/keys/release.asc
EOF"

[ceph-noarch]
name=Ceph noarch packages
baseurl=https://download.ceph.com/rpm-luminous/el7/noarch
enabled=1
gpgcheck=1
type=rpm-md
gpgkey=https://download.ceph.com/keys/release.asc
[ceph-noarch]
name=Ceph noarch packages
baseurl=https://download.ceph.com/rpm-luminous/el7/noarch
enabled=1
gpgcheck=1
type=rpm-md
gpgkey=https://download.ceph.com/keys/release.asc


#### IMPORTANT: CHANGE THE VALUE OF node_1_ip and node_2_ip (10.140.82.245 ceph-node-1) on below 2 cells!
#### node_1_ip != server1_ip!

In [14]:
eval "$login_command_1" /bin/bash << EOF
    sudo yum -y install ceph-deploy
    sudo yum -y install ntp ntpdate ntp-doc
    sudo ntpdate 0.us.pool.ntp.org
    sudo hwclock --systohc
    sudo systemctl enable ntpd.service
    sudo systemctl start ntpd.service
    sudo yum -y install openssh-server
    
    sudo bash -c "echo 10.140.82.245 ceph-node-1 >> /etc/hosts"
    sudo bash -c "echo 10.140.82.80 ceph-node-2 >> /etc/hosts"
EOF

Loaded plugins: fastestmirror, product-id, search-disabled-repos, subscription-
              : manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.dst.ca
 * extras: mirror.grid.uchicago.edu
 * updates: mirrors.cmich.edu
Resolving Dependencies
--> Running transaction check
---> Package ceph-deploy.noarch 0:2.0.1-0 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package             Arch           Version           Repository           Size
Installing:
 ceph-deploy         noarch         2.0.1-0           ceph-noarch         286 k

Transaction Summary
Install  1 Package

Total download size: 286 k
Installed size: 1.2 M
Public key for ceph-deploy-2.0.1-0.noarch.rpm is not installed
Retrieving key from https://download.ceph.com/keys/release.asc
Importing GPG key 0x460F3994:
 Userid     : "Ceph.com (release key) <se

In [15]:
eval "$login_command_2" /bin/bash << EOF
    sudo yum -y install ceph-deploy
    sudo yum -y install ntp ntpdate ntp-doc
    sudo ntpdate 0.us.pool.ntp.org
    sudo hwclock --systohc
    sudo systemctl enable ntpd.service
    sudo systemctl start ntpd.service
    sudo yum -y install openssh-server
    
    sudo bash -c "echo 10.140.82.245 ceph-node-1 >> /etc/hosts"
    sudo bash -c "echo 10.140.82.80 ceph-node-2 >> /etc/hosts"
EOF

Loaded plugins: fastestmirror, product-id, search-disabled-repos, subscription-
              : manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.gigenet.com
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
Resolving Dependencies
--> Running transaction check
---> Package ceph-deploy.noarch 0:2.0.1-0 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package             Arch           Version           Repository           Size
Installing:
 ceph-deploy         noarch         2.0.1-0           ceph-noarch         286 k

Transaction Summary
Install  1 Package

Total download size: 286 k
Installed size: 1.2 M
Public key for ceph-deploy-2.0.1-0.noarch.rpm is not installed
Retrieving key from https://download.ceph.com/keys/release.asc
Importing GPG key 0x460F3994:
 Userid     : "Ceph.com (release key) <s

#### Update ssh config

In [16]:
ssh cc@"$server1_ip" \
    "cat << EOF | sudo tee ~/.ssh/config
Host ceph-node-1
   Hostname ceph-node-1
   User cc
Host ceph-node-2
   Hostname ceph-node-2
   User cc
EOF"
ssh cc@"$server2_ip" \
    "cat << EOF | sudo tee ~/.ssh/config
Host ceph-node-1
   Hostname ceph-node-1
   User cc
Host ceph-node-2
   Hostname ceph-node-2
   User cc
EOF"

Host ceph-node-1
   Hostname ceph-node-1
   User cc
Host ceph-node-2
   Hostname ceph-node-2
   User cc
Host ceph-node-1
   Hostname ceph-node-1
   User cc
Host ceph-node-2
   Hostname ceph-node-2
   User cc


In [17]:
eval "$login_command_1" /bin/bash << EOF
    sudo ufw disable
    sudo firewall-cmd --state
    sudo setenforce 0
    sudo sed -i 's/^SELINUX=enforcing$/SELINUX=disabled/' /etc/selinux/config
    sudo yum -y install yum-plugin-priorities
EOF

eval "$login_command_2" /bin/bash << EOF
    sudo ufw disable
    sudo firewall-cmd --state
    sudo setenforce 0
    sudo sed -i 's/^SELINUX=enforcing$/SELINUX=disabled/' /etc/selinux/config
    sudo yum -y install yum-plugin-priorities
EOF

Firewall stopped and disabled on system startup
not running
setenforce: SELinux is disabled
Loaded plugins: fastestmirror, product-id, search-disabled-repos, subscription-
              : manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.dst.ca
 * extras: mirror.grid.uchicago.edu
 * updates: mirrors.cmich.edu
Resolving Dependencies
--> Running transaction check
---> Package yum-plugin-priorities.noarch 0:1.1.31-54.el7_8 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                    Arch        Version                 Repository
                                                                           Size
Installing:
 yum-plugin-priorities      noarch      1.1.31-54.el7_8         base       29 k

Transaction Summary
Install  1 Package

Total download size: 29 k
Installed size: 28 k
Running transa

#### Configure connection between node-1 and node-2 (Ceph needs passwordless access to root)

In [18]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    ssh-keygen -t rsa -b 4096  -P '' -C ceph-node-1 -f /root/.ssh/id_rsa <<< y
    cat /root/.ssh/id_rsa.pub
EOF

Generating public/private rsa key pair.
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:hYS63s1eKTYeD0x0Gy+naL+JDmOAVFLugf4KDY8wdOU ceph-node-1
The key's randomart image is:
+---[RSA 4096]----+
|  ..o. ..        |
|   =o .. .       |
| .o.oE  o +      |
|.o.o.. . o +     |
|+ o o.  S o o    |
|.* ... o . =     |
|o o...+oX +      |
| . ....*oX .     |
|  .    o= =.     |
+----[SHA256]-----+
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCpnVr4Xqtb/7//pbH+S2vE300OhYHd24OsGiE6FnxHXTq/0NarPem8iJ4ts7KgRVa0qYlVAbVWfFFrOGWm/STMIUOM+MIzi5O9henklSuAfwIi+IVqzXRcZdD8K3qw1lvW0mH54jYxP/0mZl7OD3M/Yxi65kGu7Qb+GFpA+bikkbKhNil2NbtzPMajs4Gs20RmCyTnZx1FGgYj+noD+yc2LCCmOJujy2Ll7nOlfBx95sBZKJbDFBhHkOrycwc0DD+m9FoMWkCACQVPE2H4N3OJ/l7Hx/JjGdO/q25aevj/K6pHj4KX+T+bSBlGui0AVYdMwHNcjz0oGJ7Ra8EHR4yrs91CaoHkxhEWeM/1hEWzxNK2ygyjPUHxm/RwNg8mJ9qgIDeaeo5YY7y3T2O3KmU7GDeDfUi6baFinGSbKiszktFEsqHEyYcdXb2B4IxXzhFA1h5M0haNC9oU59hQ0WFW9

#### IMPORTANT: change variable pubkeynode1 below with the id_rsa.pub content (start with ssh-rsa above)

In [19]:
pubkeynode1=`eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cat /root/.ssh/id_rsa.pub
    exit
EOF`
echo $pubkeynode1

eval "$login_command_2" /bin/bash << EOF
    echo $pubkeynode1 >> /home/cc/.ssh/authorized_keys
    
    sudo -i
    echo $pubkeynode1 >> /root/.ssh/authorized_keys
EOF

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCpnVr4Xqtb/7//pbH+S2vE300OhYHd24OsGiE6FnxHXTq/0NarPem8iJ4ts7KgRVa0qYlVAbVWfFFrOGWm/STMIUOM+MIzi5O9henklSuAfwIi+IVqzXRcZdD8K3qw1lvW0mH54jYxP/0mZl7OD3M/Yxi65kGu7Qb+GFpA+bikkbKhNil2NbtzPMajs4Gs20RmCyTnZx1FGgYj+noD+yc2LCCmOJujy2Ll7nOlfBx95sBZKJbDFBhHkOrycwc0DD+m9FoMWkCACQVPE2H4N3OJ/l7Hx/JjGdO/q25aevj/K6pHj4KX+T+bSBlGui0AVYdMwHNcjz0oGJ7Ra8EHR4yrs91CaoHkxhEWeM/1hEWzxNK2ygyjPUHxm/RwNg8mJ9qgIDeaeo5YY7y3T2O3KmU7GDeDfUi6baFinGSbKiszktFEsqHEyYcdXb2B4IxXzhFA1h5M0haNC9oU59hQ0WFW9vIDa2SzxU23k1hgjRG4YXLRvBxFAqr5wqBXbys45geT14ABUKlZyCcIbo6tCNhWPc/JVsNWxS2rKixv+Lb3vvw8lheKq9dge7eGL7z2zoKP3b6eqhr3MSRjgT5W4hmZQJeuZqdJ5BNPs2O1atURMGoZYTRcFO9wUjsS82XzRtXegWPHOhJCCyBqV4I5pg6acjMPWw93q0tACyx/Hx+DKQ== ceph-node-1


In [20]:
# Check authorized_keys of node-2 root

eval "$login_command_2" /bin/bash << EOF
    sudo -i
    cat /root/.ssh/authorized_keys
    exit
EOF

no-port-forwarding,no-agent-forwarding,no-X11-forwarding,command="echo 'Please login as the user \"cc\" rather than the user \"root\".';echo;sleep 10" ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCtBFOBerNNVEpvoe1shm/qjMTKGghqI8TtRpSbaRUt9uGhE2PVkumcI95JPKoVYUBBfhg2tlRtoNLlo7VrxfZGsw85P4hrOKqv4ZYSJoAAUvqaqN2BejxHYRZgWKHJnyhKm4LkW7gXQYMNl7OaYgAeAAP8VW6s7w3QyjNFUKYoZ4aaSRN5u9ztitSzNbbSCM6kl5MDKXTHUfsV8UO4Sm0Z2s725kEHT0D/JP0jOag+IP3sOek14Jee3qGsGnYprwTV2uHRbXOxBo1SvXE6WgugEc9I6VpILAXxZaa47dJs1gxXBSfg73Bpsfua+9yCp7J70Lzlo6MFzd3PZanmxAA0w3g/nhO4gIuUoQtMIFEnGBMzt6LyT+6ubPgj4aTVkthQqLPmaLNlMrXvBwHF/hJUzAMvDw3zK3qQZTZIOfRtQQzrTQkXS0rxxLl+G/0eziYf0CQnhTkCvH4ToTyxaUgUAjppYDyTEe4mnBaehXiALtLdZFuwoVBatHHjVIboZh0= rani@LAPTOP-6S2PSCIK
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCpnVr4Xqtb/7//pbH+S2vE300OhYHd24OsGiE6FnxHXTq/0NarPem8iJ4ts7KgRVa0qYlVAbVWfFFrOGWm/STMIUOM+MIzi5O9henklSuAfwIi+IVqzXRcZdD8K3qw1lvW0mH54jYxP/0mZl7OD3M/Yxi65kGu7Qb+GFpA+bikkbKhNil2NbtzPMajs4Gs20RmCyTnZx1FGgYj+noD+yc2LCCmOJujy2Ll7nOlfBx95sB

#### Alternatively, can use ssh-copy-id to configure connection between node-1 and node-2

In [21]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    ssh-keyscan ceph-node-1 ceph-node-2 >> ~/.ssh/known_hosts
    ssh-copy-id ceph-node-1
    ssh-copy-id ceph-node-2
    exit
EOF

# ceph-node-1:22 SSH-2.0-OpenSSH_7.4
# ceph-node-1:22 SSH-2.0-OpenSSH_7.4
# ceph-node-1:22 SSH-2.0-OpenSSH_7.4
# ceph-node-2:22 SSH-2.0-OpenSSH_7.4
# ceph-node-2:22 SSH-2.0-OpenSSH_7.4
# ceph-node-2:22 SSH-2.0-OpenSSH_7.4
/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/root/.ssh/id_rsa.pub"
/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys
Permission denied (publickey,gssapi-keyex,gssapi-with-mic).
/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/root/.ssh/id_rsa.pub"
/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.
		(if you think this is a mistake, you may want to use -f option)



## 5. Deploy Ceph

In [22]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    sudo mkdir -p /mnt/extra/ceph_cluster
    cd /mnt/extra/ceph_cluster
    echo yes | ceph-deploy new ceph-node-1 ceph-node-2
    exit
EOF

[ceph_deploy.conf][DEBUG ] found configuration file at: /root/.cephdeploy.conf
[ceph_deploy.cli][INFO  ] Invoked (2.0.1): /bin/ceph-deploy new ceph-node-1 ceph-node-2
[ceph_deploy.cli][INFO  ] ceph-deploy options:
[ceph_deploy.cli][INFO  ]  username                      : None
[ceph_deploy.cli][INFO  ]  func                          : <function new at 0x7fe4d942bf50>
[ceph_deploy.cli][INFO  ]  verbose                       : False
[ceph_deploy.cli][INFO  ]  overwrite_conf                : False
[ceph_deploy.cli][INFO  ]  quiet                         : False
[ceph_deploy.cli][INFO  ]  cd_conf                       : <ceph_deploy.conf.cephdeploy.Conf instance at 0x7fe4d942a4d0>
[ceph_deploy.cli][INFO  ]  cluster                       : ceph
[ceph_deploy.cli][INFO  ]  ssh_copykey                   : True
[ceph_deploy.cli][INFO  ]  mon                           : ['ceph-node-1', 'ceph-node-2']
[ceph_deploy.cli][INFO  ]  public_network                : None
[ceph_deploy.cli][INFO  ]  ceph_

In [24]:
# Check ceph.conf generated

eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra/ceph_cluster
    cat ceph.conf
    exit
EOF

[global]
fsid = b6498150-c23d-4428-b8f4-e3d924768878
mon_initial_members = ceph-node-1, ceph-node-2
mon_host = 10.140.82.245,10.140.82.80
auth_cluster_required = cephx
auth_service_required = cephx
auth_client_required = cephx



#### Patch

In [25]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    sudo mv /etc/yum.repos.d/ceph.repo /etc/yum.repos.d/ceph-deploy.repo
    exit
EOF

eval "$login_command_2" /bin/bash << EOF
    sudo -i
    sudo mv /etc/yum.repos.d/ceph.repo /etc/yum.repos.d/ceph-deploy.repo
    exit
EOF

#### Install Ceph and configure for loop devices

In [26]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra/ceph_cluster
    ceph-deploy install --release luminous ceph-node-1 ceph-node-2
    exit
    
    sudo sed -i "s|return TYPE == 'disk'|return TYPE == 'disk' or TYPE == 'loop' or TYPE == 'part'|g" /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py
    cat /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py | grep "TYPE == 'loop'"
EOF

[ceph_deploy.conf][DEBUG ] found configuration file at: /root/.cephdeploy.conf
[ceph_deploy.cli][INFO  ] Invoked (2.0.1): /bin/ceph-deploy install --release luminous ceph-node-1 ceph-node-2
[ceph_deploy.cli][INFO  ] ceph-deploy options:
[ceph_deploy.cli][INFO  ]  verbose                       : False
[ceph_deploy.cli][INFO  ]  testing                       : None
[ceph_deploy.cli][INFO  ]  cd_conf                       : <ceph_deploy.conf.cephdeploy.Conf instance at 0x7fc3254a42d8>
[ceph_deploy.cli][INFO  ]  cluster                       : ceph
[ceph_deploy.cli][INFO  ]  dev_commit                    : None
[ceph_deploy.cli][INFO  ]  install_mds                   : False
[ceph_deploy.cli][INFO  ]  stable                        : None
[ceph_deploy.cli][INFO  ]  default_release               : False
[ceph_deploy.cli][INFO  ]  username                      : None
[ceph_deploy.cli][INFO  ]  adjust_repos                  : True
[ceph_deploy.cli][INFO  ]  func                          : <fun

In [27]:
eval "$login_command_2" /bin/bash << EOF
    sudo -i
    sudo sed -i "s|return TYPE == 'disk'|return TYPE == 'disk' or TYPE == 'loop' or TYPE == 'part'|g" /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py
    cat /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py | grep "TYPE == 'loop'"
EOF

        return TYPE == 'disk' or TYPE == 'loop' or TYPE == 'part'


#### Copy config file to all nodes, create manager daemon, and create OSDs

In [28]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra/ceph_cluster
    ceph-deploy mon create-initial
    ceph-deploy admin ceph-node-1 ceph-node-2
    ceph-deploy mgr create ceph-node-1
    ceph-deploy mds create ceph-node-1 ceph-node-2
    sudo ceph status
    
    ceph-deploy osd create --data /dev/loop1 ceph-node-1
    ceph-deploy osd create --data /dev/loop2 ceph-node-1
    ceph-deploy osd create --data /dev/loop3 ceph-node-1
    ceph-deploy osd create --data /dev/loop1 ceph-node-2
    ceph-deploy osd create --data /dev/loop2 ceph-node-2
    ceph-deploy osd create --data /dev/loop3 ceph-node-2
    sudo ceph osd pool create mypool 100 100 replicated
    ceph tell mon.\* injectargs '--mon-allow-pool-delete=true'
EOF

[ceph_deploy.conf][DEBUG ] found configuration file at: /root/.cephdeploy.conf
[ceph_deploy.cli][INFO  ] Invoked (2.0.1): /bin/ceph-deploy mon create-initial
[ceph_deploy.cli][INFO  ] ceph-deploy options:
[ceph_deploy.cli][INFO  ]  username                      : None
[ceph_deploy.cli][INFO  ]  verbose                       : False
[ceph_deploy.cli][INFO  ]  overwrite_conf                : False
[ceph_deploy.cli][INFO  ]  subcommand                    : create-initial
[ceph_deploy.cli][INFO  ]  quiet                         : False
[ceph_deploy.cli][INFO  ]  cd_conf                       : <ceph_deploy.conf.cephdeploy.Conf instance at 0x7f4609b06170>
[ceph_deploy.cli][INFO  ]  cluster                       : ceph
[ceph_deploy.cli][INFO  ]  func                          : <function mon at 0x7f4609d5e578>
[ceph_deploy.cli][INFO  ]  ceph_conf                     : None
[ceph_deploy.cli][INFO  ]  default_release               : False
[ceph_deploy.cli][INFO  ]  keyrings                     

#### Test creating pool, testing health

In [29]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra/ceph_cluster
    sudo ceph osd pool application enable mypool rgw
    sudo ceph -s
    sudo ceph health
    
    sudo ceph mgr module enable dashboard
    sudo ceph mgr services
    
    # Clean Pool
    ceph osd pool delete mypool mypool --yes-i-really-really-mean-it
    
    exit
EOF

enabled application 'rgw' on pool 'mypool'
  cluster:
    id:     b6498150-c23d-4428-b8f4-e3d924768878
    health: HEALTH_WARN
            Degraded data redundancy: 100 pgs undersized
            clock skew detected on mon.ceph-node-1
 
  services:
    mon: 2 daemons, quorum ceph-node-2,ceph-node-1
    mgr: ceph-node-1(active)
    osd: 6 osds: 6 up, 6 in
 
  data:
    pools:   1 pools, 100 pgs
    objects: 0 objects, 0B
    usage:   6.02GiB used, 140GiB / 146GiB avail
    pgs:     100 active+undersized
 
HEALTH_WARN Degraded data redundancy: 100 pgs undersized; clock skew detected on mon.ceph-node-1
{}
pool 'mypool' removed


## 6. Clone cortx-bench-extra repo

In [30]:
eval "$login_command_1" /bin/bash << EOF
    # Change the email
    # Change the "rannnayy" with your github username
    # Set passwordless github push/pull 
    sudo -i
    cd /mnt/extra
    git config --global user.email "rani.api3939@gmail.com"
    git config --global user.name "rannnayy"
    git config --global credential.helper store         # store pass in ~/.git-credentials as plain text format.
    exit
EOF

In [31]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra 
    git clone https://rannnayy@github.com/daniarherikurniawan/cortx-bench-extra.git
    exit
    
    # Run clear cache loop  
    # cd /mnt/extra/cortx-bench-extra/script  
    # sudo ./free_page_cache.sh 0.25
EOF

Cloning into 'cortx-bench-extra'...


## 7. Configure Librados, Benchmark Librados with Rados Bench, Bonnie++, and Python Code

In [32]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    sudo yum install python-rados
    echo 'keyring = /mnt/extra/ceph_cluster/ceph.client.admin.keyring' > ceph.conf; chmod go-rw ceph.conf

    # Benchmark with RADOS BENCH
    # Ref: https://youtu.be/7puwMpv1yh8, https://www.sebastien-han.fr/blog/2012/08/26/ceph-benchmarks/
    
    # Create new storage pool
    ceph osd pool create testbench 100 100
    # Generate data
    rados bench -p testbench 5 write --no-cleanup
    
    exit
EOF

Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package 2:python-rados-12.2.13-0.el7.x86_64 already installed and latest version
Nothing to do
pool 'testbench' created
hints = 1
Maintaining 16 concurrent writes of 4194304 bytes to objects of size 4194304 for up to 5 seconds or 0 objects
Object prefix: benchmark_data_ceph-node-1.novalocal_94529
  sec Cur ops   started  finished  avg MB/s  cur MB/s last lat(s)  avg lat(s)
    0       0         0         0         0         0           -           0
    1      16        16         0         0         0           -           0
    2      16       116      

In [43]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    
    # Create new storage pool
    ceph osd pool create testbench 100 100
    
    # Benchmarking Write
     # 100s all
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 4K
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 8K
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 16K
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 32K
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 64K
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 128K
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 256K
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rados bench -p testbench 100 write -t 1 -b 512K
        
    # Benchmarking Read
    rados bench -p testbench 50 rand -t 1
    
    # Clean Pool
    ceph osd pool delete testbench testbench --yes-i-really-really-mean-it
    
    exit
EOF

pool 'testbench' created
4K
tee: /proc/sys/vm/drop_caches: Invalid argument
hints = 1
Maintaining 1 concurrent writes of 4096 bytes to objects of size 4096 for up to 100 seconds or 0 objects
Object prefix: benchmark_data_ceph-node-1.novalocal_95013
  sec Cur ops   started  finished  avg MB/s  cur MB/s last lat(s)  avg lat(s)
    0       0         0         0         0         0           -           0
    1       1        44        43   0.16796  0.167969  0.00132899   0.0232331
    2       1       506       505  0.986258   1.80469  0.00186827  0.00395729
    3       1       951       950   1.23689   1.73828  0.00217319  0.00315529
    4       1      1363      1362   1.32996   1.60938  0.00274803  0.00293405
    5       1      1757      1756   1.37175   1.53906  0.00191308  0.00284485
    6       1      2163      2162   1.40742   1.58594  0.00273172  0.00277251
    7       1      2572      2571   1.43458   1.59766  0.00239068  0.00272068
    8       1      2997      2996   1.46276   1.6

In [44]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    
    # Create new storage pool
    ceph osd pool create testbench 100 100
    
    # Benchmark with Bonnie++
      # Ref: https://www.jamescoyle.net/how-to/599-benchmark-disk-io-with-dd-and-bonnie
      # https://www.sebastien-han.fr/blog/2012/08/26/ceph-benchmarks/

      # Install bonnie++
      sudo yum -y install bonnie++

      cd /mnt/extra/ceph_cluster
      
      # 4K
      echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 2K -s 4K -n 1 -f -b -u root -m Bonnie4K
      # 8K
      echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 4K -s 8K -n 1 -f -b -u root -m Bonnie8K
      # 16K
      echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 8K -s 16K -n 1 -f -b -u root -m Bonnie16K
      # 32K
      echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 16K -s 32K -n 1 -f -b -u root -m Bonnie32K
      # 64K
      echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 32K -s 64K -n 1 -f -b -u root -m Bonnie64K
      # 128K
      echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 64K -s 128K -n 1 -f -b -u root -m Bonnie128K
      # 256K
      echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 128K -s 256K -n 1 -f -b -u root -m Bonnie256K
      # 512K
      echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
      bonnie++ -d /mnt/extra/ -r 256K -s 512K -n 1 -f -b -u root -m Bonnie512K

    # Clean Pool
    ceph osd pool delete testbench testbench --yes-i-really-really-mean-it
    
    exit
EOF

pool 'testbench' created
Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package bonnie++.x86_64 0:1.97.3-1.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package           Arch            Version                  Repository     Size
Installing:
 bonnie++          x86_64          1.97.3-1.el7             epel           79 k

Transaction Summary
Install  1 Package

Total download size: 79 k
Installed size: 190 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  I

In [54]:
# Check if Instance has python3 installed
# Install Dependencies for Benchmark with Python Code

eval "$login_command_1" /bin/bash << EOF
    sudo -i
    python3 --version
    echo y | sudo yum install python3-rados
    exit
EOF

Python 3.6.8
Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package 2:python36-rados-12.2.13-0.el7.x86_64 already installed and latest version
Nothing to do


In [60]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    
    # Create new storage pool
    ceph osd pool create mypool 100 100
    
    # Benchmark with Python Code
    cd /mnt/extra/cortx-bench-extra/librados-clients/
    python3 ceph-client-bench.py
    
    # Clean Pool
    ceph osd pool delete mypool mypool --yes-i-really-really-mean-it
    
    exit
EOF

pool 'mypool' created
Created cluster handle.
Connected to the cluster.


I/O Context and Object Operations

Creating a context for the 'mypool' pool
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
write done
read done
delete done
Write Latency
4096 : 0.002159205728985853
8192 : 0.0022795713294876804
16384 : 0.0023125487094894197
32768 : 0.0024945674519831302
65536 : 0.0027261736560062672
131072 : 0.003107458164006857
262144 : 0.003986656674498135
524288 : 0.0051151641735045815
1048576 : 0.007607427247505711
2097152 : 0.013561914065006932
4194304 : 0.025025193638502516

Read Latency
4096 : 0.0004551075749909614
8192 : 0.0005028642514944295
16384 : 0.0004922012844949677
32768 : 0.00044842209149737757
65536 : 0

## 8. Setup Rados Block Devices (RBD), Benchmark with RBD Bench and Bonnie++

In [61]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cd /mnt/extra/ceph_cluster

    # the pool 'mypool' already registered with rgw, so make another pool
    sudo ceph osd pool create rbdpool 100 100 replicated
    sudo modprobe rbd
    rbd pool init rbdpool

    # Create Block Device Image
    rbd create rbdimage --size 1024 --pool rbdpool --image-feature layering
    # Use admin user
    sudo rbd map rbdimage --pool rbdpool --name client.admin

    sudo mkfs.ext4 -m0 /dev/rbd/rbdpool/rbdimage
    sudo mkdir /mnt/extra/ceph-block-device
    sudo mount /dev/rbd/rbdpool/rbdimage /mnt/extra/ceph-block-device
    cd /mnt/extra/ceph-block-device

    # https://docs.ceph.com/en/quincy/man/8/rbd/
    # bench –io-type <read | write | readwrite | rw> [–io-size size-in-B/K/M/G/T] [–io-threads num-ios-in-flight] [–io-total size-in-B/K/M/G/T] [–io-pattern seq | rand] [–rw-mix-read read proportion in readwrite] image-spec
    # Generate a series of IOs to the image and measure the IO throughput and latency. If no suffix is given, unit B is assumed for both –io-size and –io-total. Defaults are: –io-size 4096, –io-threads 16, –io-total 1G, –io-pattern seq, –rw-mix-read 50.
    
    # For sequential, write
    echo "sequential, write"
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 4K --io-threads 1 --io-total 4G --io-pattern seq
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 8K --io-threads 1 --io-total 4G --io-pattern seq
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 16K --io-threads 1 --io-total 4G --io-pattern seq
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 32K --io-threads 1 --io-total 4G --io-pattern seq
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 64K --io-threads 1 --io-total 4G --io-pattern seq
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 128K --io-threads 1 --io-total 4G --io-pattern seq
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 256K --io-threads 1 --io-total 4G --io-pattern seq
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 512K --io-threads 1 --io-total 4G --io-pattern seq
    
    # For random, write
    echo "random, write"
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 4K --io-threads 1 --io-total 4G --io-pattern rand
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 8K --io-threads 1 --io-total 4G --io-pattern rand
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 16K --io-threads 1 --io-total 4G --io-pattern rand
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 32K --io-threads 1 --io-total 4G --io-pattern rand
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 64K --io-threads 1 --io-total 4G --io-pattern rand
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 128K --io-threads 1 --io-total 4G --io-pattern rand
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 256K --io-threads 1 --io-total 4G --io-pattern rand
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type write rbdimage --pool=rbdpool --io-size 512K --io-threads 1 --io-total 4G --io-pattern rand
    
    # For sequential, read
    echo "sequential, read"
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 4K --io-threads 1 --io-total 4G --io-pattern seq
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 8K --io-threads 1 --io-total 4G --io-pattern seq
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 16K --io-threads 1 --io-total 4G --io-pattern seq
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 32K --io-threads 1 --io-total 4G --io-pattern seq
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 64K --io-threads 1 --io-total 4G --io-pattern seq
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 128K --io-threads 1 --io-total 4G --io-pattern seq
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 256K --io-threads 1 --io-total 4G --io-pattern seq
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 512K --io-threads 1 --io-total 4G --io-pattern seq

    # For random, read
    echo "random, read"
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 4K --io-threads 1 --io-total 4G --io-pattern rand
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 8K --io-threads 1 --io-total 4G --io-pattern rand
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 16K --io-threads 1 --io-total 4G --io-pattern rand
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 32K --io-threads 1 --io-total 4G --io-pattern rand
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 64K --io-threads 1 --io-total 4G --io-pattern rand
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 128K --io-threads 1 --io-total 4G --io-pattern rand
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 256K --io-threads 1 --io-total 4G --io-pattern rand
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        rbd bench --io-type read rbdimage --pool=rbdpool --io-size 512K --io-threads 1 --io-total 4G --io-pattern rand

    # Benchmark RBD using Bonnie++
    echo "RBD Bonnie++"
        # 4K
        echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 2K -s 4K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 8K
        echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 4K -s 8K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 16K
        echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 8K -s 16K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 32K
        echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 16K -s 32K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 64K
        echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 32K -s 64K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 128K
        echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 64K -s 128K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 256K
        echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 128K -s 256K -n 1 -f -b -u root:root -m BonnieRBD$size
        # 512K
        echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
        bonnie++ -d /mnt/extra/ -r 256K -s 512K -n 1 -f -b -u root:root -m BonnieRBD$size

    # Clean Pool
    ceph osd pool delete rbdpool rbdpool --yes-i-really-really-mean-it
    
    exit
EOF

pool 'rbdpool' created
/dev/rbd0
mke2fs 1.42.9 (28-Dec-2013)
Discarding device blocks:   4096/26214            done                            
Filesystem label=
OS type: Linux
Block size=4096 (log=2)
Fragment size=4096 (log=2)
Stride=1024 blocks, Stripe width=1024 blocks
65536 inodes, 262144 blocks
0 blocks (0.00%) reserved for the super user
First data block=0
Maximum filesystem blocks=268435456
8 block groups
32768 blocks per group, 32768 fragments per group
8192 inodes per group
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376

Allocating group tables: 0/  done                            
Writing inode tables: 0/  done                            
Creating journal (8192 blocks): done
Writing superblocks and filesystem accounting information: 0/  done

sequential, write
4K
tee: /proc/sys/vm/drop_caches: Invalid argument
bench  type write io_size 4096 io_threads 1 bytes 4294967296 pattern sequential
  SEC       OPS   OPS/SEC   BYTES/SEC
    1     29437  29438.32  12

## 9. Setup Rados Gateway, Benchmark with Hotsauce Bench

In [62]:
eval "$login_command_1" /bin/bash << EOF
    sudo yum -y install ceph-radosgw
    sudo yum -y install radosgw-agent
    sudo ceph-authtool --create-keyring /etc/ceph/ceph.client.radosgw.keyring
    sudo chmod +r /etc/ceph/ceph.client.radosgw.keyring
    sudo ceph-authtool /etc/ceph/ceph.client.radosgw.keyring -n client.radosgw.ceph-node-1 --gen-key
    sudo ceph-authtool -n client.radosgw.ceph-node-1 --cap osd 'allow rwx' --cap mon 'allow rwx' /etc/ceph/ceph.client.radosgw.keyring
    sudo ceph -k /etc/ceph/ceph.client.admin.keyring auth add client.radosgw.ceph-node-1 -i /etc/ceph/ceph.client.radosgw.keyring
EOF

Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package 2:ceph-radosgw-12.2.13-0.el7.x86_64 already installed and latest version
Nothing to do
Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Resolving Dependencies
--> 

In [63]:
ssh cc@"$server1_ip" \
    "cat <<EOF | sudo tee -a /etc/ceph/ceph.conf
[client.radosgw.ceph-node-1]
host = ceph-node-1
keyring = /etc/ceph/ceph.client.radosgw.keyring
rgw socket path = ""
log file = /var/log/radosgw/client.radosgw.ceph-node-1.log
rgw frontends = \"civetweb port=8080\"
rgw print continue = false
EOF"

[client.radosgw.ceph-node-1]
host = ceph-node-1
keyring = /etc/ceph/ceph.client.radosgw.keyring
rgw socket path = 
log file = /var/log/radosgw/client.radosgw.ceph-node-1.log
rgw frontends = "civetweb port=8080"
rgw print continue = false


In [64]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    cat /etc/ceph/ceph.conf
    exit
EOF

[global]
fsid = b6498150-c23d-4428-b8f4-e3d924768878
mon_initial_members = ceph-node-1, ceph-node-2
mon_host = 10.140.82.245,10.140.82.80
auth_cluster_required = cephx
auth_service_required = cephx
auth_client_required = cephx

[client.radosgw.ceph-node-1]
host = ceph-node-1
keyring = /etc/ceph/ceph.client.radosgw.keyring
rgw socket path = 
log file = /var/log/radosgw/client.radosgw.ceph-node-1.log
rgw frontends = "civetweb port=8080"
rgw print continue = false


In [65]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    ceph-deploy --overwrite-conf config pull ceph-node-1
    ceph-deploy --overwrite-conf config push ceph-node-2
    sudo mkdir -p /var/lib/ceph/radosgw/ceph-radosgw.ceph-node-1
    sudo mkdir -p /var/log/radosgw/
    sudo chown cc /var/run/ceph
    sudo chown cc /var/log/radosgw/
    
    sudo systemctl start ceph-radosgw@radosgw.ceph-node-1
    sudo systemctl status ceph-radosgw@radosgw.ceph-node-1

    sudo firewall-cmd --state
    exit
EOF

[ceph_deploy.conf][DEBUG ] found configuration file at: /root/.cephdeploy.conf
[ceph_deploy.cli][INFO  ] Invoked (2.0.1): /bin/ceph-deploy --overwrite-conf config pull ceph-node-1
[ceph_deploy.cli][INFO  ] ceph-deploy options:
[ceph_deploy.cli][INFO  ]  username                      : None
[ceph_deploy.cli][INFO  ]  verbose                       : False
[ceph_deploy.cli][INFO  ]  overwrite_conf                : True
[ceph_deploy.cli][INFO  ]  subcommand                    : pull
[ceph_deploy.cli][INFO  ]  quiet                         : False
[ceph_deploy.cli][INFO  ]  cd_conf                       : <ceph_deploy.conf.cephdeploy.Conf instance at 0x7f74dcc51f80>
[ceph_deploy.cli][INFO  ]  cluster                       : ceph
[ceph_deploy.cli][INFO  ]  client                        : ['ceph-node-1']
[ceph_deploy.cli][INFO  ]  func                          : <function config at 0x7f74dce77d70>
[ceph_deploy.cli][INFO  ]  ceph_conf                     : None
[ceph_deploy.cli][INFO  ]  defau

: 7

In [66]:
#### Wait for some time (~5s) before running this cell

eval "$login_command_1" /bin/bash << EOF
    sudo -i
    curl ceph-node-1:8080
    exit
EOF

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   214    0   214    0 <?xml version="1.0" encoding="UTF-8"?><ListAllMyBucketsResult xmlns="http://s3.amazonaws.com/doc/2006-03-01/"><Owner><ID>anonymous</ID><DisplayName></DisplayName></Owner><Buckets></Buckets></ListAllMyBucketsResult>    0  32726      0 --:--:-- --:--:-- --:--:-- 35666


In [67]:
eval "$login_command_1" /bin/bash << EOF
    # Create a radosgw user for S3 access
        sudo radosgw-admin user create --uid="testuser" --display-name="First User"
        sudo radosgw-admin user list
        sudo radosgw-admin user info --uid=testuser

    # Create a Swift user
        sudo radosgw-admin subuser create --uid=testuser --subuser=testuser:swift --access=full
EOF

{
    "user_id": "testuser",
    "display_name": "First User",
    "email": "",
    "suspended": 0,
    "max_buckets": 1000,
    "auid": 0,
    "subusers": [],
    "keys": [
        {
            "user": "testuser",
            "access_key": "NQQ5X7SH9Z963DEBHY74",
            "secret_key": "mB9dTQ3ouu19v6il6B8NzaIRgplcuMpsySub2mf7"
        }
    ],
    "swift_keys": [],
    "caps": [],
    "op_mask": "read, write, delete",
    "default_placement": "",
    "placement_tags": [],
    "bucket_quota": {
        "enabled": false,
        "check_on_raw": false,
        "max_size": -1,
        "max_size_kb": 0,
        "max_objects": -1
    },
    "user_quota": {
        "enabled": false,
        "check_on_raw": false,
        "max_size": -1,
        "max_size_kb": 0,
        "max_objects": -1
    },
    "temp_url_keys": [],
    "type": "rgw"
}

[
    "testuser"
]
{
    "user_id": "testuser",
    "display_name": "First User",
    "email": "",
    "suspended": 0,
    "max_buckets": 1000,
    "

In [68]:
eval "$login_command_1" /bin/bash << EOF
    # get the "access_key" and "secret_key" from above cell's output
    
    # Create the secret key:
        sudo radosgw-admin key create --subuser=testuser:swift --key-type=swift --gen-secret
EOF

{
    "user_id": "testuser",
    "display_name": "First User",
    "email": "",
    "suspended": 0,
    "max_buckets": 1000,
    "auid": 0,
    "subusers": [
        {
            "id": "testuser:swift",
            "permissions": "full-control"
        }
    ],
    "keys": [
        {
            "user": "testuser",
            "access_key": "NQQ5X7SH9Z963DEBHY74",
            "secret_key": "mB9dTQ3ouu19v6il6B8NzaIRgplcuMpsySub2mf7"
        }
    ],
    "swift_keys": [
        {
            "user": "testuser:swift",
            "secret_key": "KfkTKv5mxOnkKaT5OSDQWR9ZluepAxAMbfAlBet3"
        }
    ],
    "caps": [],
    "op_mask": "read, write, delete",
    "default_placement": "",
    "placement_tags": [],
    "bucket_quota": {
        "enabled": false,
        "check_on_raw": false,
        "max_size": -1,
        "max_size_kb": 0,
        "max_objects": -1
    },
    "user_quota": {
        "enabled": false,
        "check_on_raw": false,
        "max_size": -1,
        "max_size_k

In [73]:
eval "$login_command_1" /bin/bash << EOF
    # get the "secret_key" under "swift_keys" from above cell's output
    
    sudo yum -y install python-boto
    sudo yum install net-tools
EOF

Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package python-boto-2.25.0-2.el7.centos.noarch is obsoleted by python2-boto-2.45.0-3.el7.noarch which is already installed
Nothing to do
Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priori

In [76]:
export PORT=8080
export IP=`eval "$login_command_1" /bin/bash << EOF
    sudo -i
    ifconfig | grep "inet 10" | awk '{print $2}'
    exit
EOF`
echo $IP
export blockSizeArr=("4K" "8K" "16K" "32K" "64K" "128K" "256K" "512K" )
export access_key="NQQ5X7SH9Z963DEBHY74"
export secret_key="mB9dTQ3ouu19v6il6B8NzaIRgplcuMpsySub2mf7"
export swiftsecretkey="KfkTKv5mxOnkKaT5OSDQWR9ZluepAxAMbfAlBet3"

inet 10.140.82.245 netmask 255.255.252.0 broadcast 10.140.83.255


In [81]:
#### IMPORTANT: change 'pQD6Feuy7sn9yn2BfjQDmQwXxVnsNv1nslV7j2Au' to swift secret key

eval "$login_command_1" /bin/bash << EOF
    sudo -i
    sudo yum -y install python-setuptools
    pip install setuptools
    pip install pbr
    pip install python-swiftclient
    
    # run test 
    # swift -A http://10.140.82.245:8080/auth/1.0 -U testuser:swift -K 'KfkTKv5mxOnkKaT5OSDQWR9ZluepAxAMbfAlBet3' list
    exit
EOF

Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package python-setuptools-0.9.8-7.el7.noarch already installed and latest version
Nothing to do
You are using pip version 8.1.2, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/94/3a/39018dcc045c6f26230792ae2ee609acd41a8be1f4910c25228439973cf0/python-swiftclient-4.0.1.tar.gz
    Complete output f

: 1

In [82]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i
    sudo yum -y install jq
    sudo yum install -y go

    go env 
        # GOROOT="/usr/lib/golang"
        # GOPATH="/root/go"

    which go
    sudo go install github.com/aws/aws-sdk-go@latest
    sudo find / -type d -name 'aws-sdk-go'
        # /root/go/pkg/mod/cache/download/github.com/aws/aws-sdk-go

    # Install hsbench
    sudo go install github.com/markhpc/hsbench@latest 
    sudo find / -name 'hsbench'
        # /root/go/pkg/mod/cache/download/github.com/markhpc/hsbench

    sudo ls /root/go/bin
        # hsbench
    exit
EOF

Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Package jq-1.6-2.el7.x86_64 already installed and latest version
Nothing to do
Loaded plugins: fastestmirror, priorities, product-id, search-disabled-repos,
              : subscription-manager

This system is not registered with an entitlement server. You can use subscription-manager to register.

Loading mirror speeds from cached hostfile
 * base: mirrors.cmich.edu
 * epel: mirror.usi.edu
 * extras: mirror.grid.uchicago.edu
 * updates: ftp.ussg.iu.edu
27 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transact

In [85]:
export access_key="NQQ5X7SH9Z963DEBHY74"
export secret_key="mB9dTQ3ouu19v6il6B8NzaIRgplcuMpsySub2mf7"
export swiftsecretkey="KfkTKv5mxOnkKaT5OSDQWR9ZluepAxAMbfAlBet3"

In [86]:
eval "$login_command_1" /bin/bash << EOF
    echo $access_key
    echo $secret_key
    echo $swiftsecretkey
EOF

NQQ5X7SH9Z963DEBHY74
mB9dTQ3ouu19v6il6B8NzaIRgplcuMpsySub2mf7
KfkTKv5mxOnkKaT5OSDQWR9ZluepAxAMbfAlBet3


In [87]:
eval "$login_command_1" /bin/bash << EOF
    sudo -i

    echo $IP:$PORT

    cd /root/go/bin
    # 4K
    echo "4K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 4K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 4K.log
    cat 4K.log | jq | grep TOTAL -A 8
    # 8K
    echo "8K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 8K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 8K.log
    cat 8K.log | jq | grep TOTAL -A 8
    # 16K
    echo "16K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 16K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 16K.log
    cat 16K.log | jq | grep TOTAL -A 8
    # 32K
    echo "32K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 32K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 32K.log
    cat 32K.log | jq | grep TOTAL -A 8
    # 64K
    echo "64K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 64K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 64K.log
    cat 64K.log | jq | grep TOTAL -A 8
    # 128K
    echo "128K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 128K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 128K.log
    cat 128K.log | jq | grep TOTAL -A 8
    # 256K
    echo "256K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 256K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 256K.log
    cat 256K.log | jq | grep TOTAL -A 8
    # 512K
    echo "512K" | sudo tee /proc/sys/vm/drop_caches && sudo sync
    ./hsbench -a $access_key -s $secret_key -u http://10.140.82.245:8080  -z 512K -t 1 -n 10000 -b 100 -d -1 -m cxipgdcx -ri 10 -j 512K.log
    cat 512K.log | jq | grep TOTAL -A 8
    
    exit
EOF

inet 10.140.82.245 netmask 255.255.252.0 broadcast 10.140.83.255:8080
4K
tee: /proc/sys/vm/drop_caches: Invalid argument
2022/08/25 10:40:39 Hotsauce S3 Benchmark Version 0.1
2022/08/25 10:40:39 Parameters:
2022/08/25 10:40:39 url=http://10.140.82.245:8080
2022/08/25 10:40:39 object_prefix=
2022/08/25 10:40:39 bucket_prefix=hotsauce-bench
2022/08/25 10:40:39 region=us-east-1
2022/08/25 10:40:39 modes=cxipgdcx
2022/08/25 10:40:39 output=
2022/08/25 10:40:39 json_output=4K.log
2022/08/25 10:40:39 max_keys=1000
2022/08/25 10:40:39 object_count=10000
2022/08/25 10:40:39 bucket_count=100
2022/08/25 10:40:39 duration=-1
2022/08/25 10:40:39 threads=1
2022/08/25 10:40:39 loops=1
2022/08/25 10:40:39 size=4K
2022/08/25 10:40:39 interval=10.000000
2022/08/25 10:40:39 Running Loop 0 BUCKET CLEAR TEST
2022/08/25 10:40:39 Loop: 0, Int: TOTAL, Dur(s): 0.0, Mode: BCLR, Ops: 0, MB/s: 0.00, IO/s: 0, Lat(ms): [ min: 0.0, avg: 0.0, 99%: 0.0, max: 0.0 ], Slowdowns: 0
2022/08/25 10:40:39 Running Loop 0 BUCK